# Text2SQL

: KoGPT2를 사용하여 사용자의 자연어 문장을 입력받아 SQL문을 제공하는 모델을 만들어보자

Input:

"동대문구 20대 남자의 수를 출력하고싶어"

Output:

```
SELECT COUNT(*) AS num_of_20_male
FROM people table
WHERE location = '동대문구'   
AND age BETWEENT 20 AND 29 AND gender='남자';
```



# 00. Library import

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
import numpy as np
import random
import pandas as pd

from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# project 폴더로 colab 위치 이동
%cd drive/MyDrive/Colab Notebooks/Text2SQL

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Text2SQL


# 01. Word Dictionary

label_file : TEXT_NL2SQL_label_seouldata_healthcare.json
→ DB_id, Query, utterance


Original_file : seouldata_healthcare_db_annotation.json
→ DB_id, Table_names_original, Column_names_original, Column_types

In [ ]:
import json
file_path = "healthcare_dataset/Training/라벨링데이터/TEXT_NL2SQL_label_seouldata_healthcare.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

df = pd.DataFrame(data['data'])
df.head()

,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols
0,seouldata_healthcare_455,Whr_0001,easy,BR03,SELECT ADMDNG_NM FROM SEOUL_PUBLIC_HYGIENE_BIZ...,태창위생지의 행정동 이름이 뭐야,"[{'token': '태창위생지', 'start': 0, 'column_index'...","[{'token': '행정동 이름', 'start': 7, 'column_index..."
1,seouldata_healthcare_455,Wht_0001,medium,BR04,"SELECT UPSO_SITE_TELNO, TRDP_AREA FROM SEOUL_P...",2010년에 폐업한 영업장의 면적과 소재지 전화번호를 나타내줘,[],"[{'token': '영업장의 면적', 'start': 11, 'column_ind..."
2,seouldata_healthcare_455,Hch_0001,medium,BR08,"SELECT UPSO_NM, TRDP_AREA FROM SEOUL_PUBLIC_HY...",행정동명이 용신동인 업소의 이름과 영업장 면적을 영업자 시작일이 최근인 순서대로 찾아줘,"[{'token': '용신동', 'start': 6, 'column_index': ...","[{'token': '업소의 이름', 'start': 11, 'column_inde..."
3,seouldata_healthcare_455,Whn_0001,medium,BR02,SELECT DCB_YMD FROM SEOUL_PUBLIC_HYGIENE_BIZ W...,영업장 면적이 50 이하이거나 100 이상인 곳의 폐업 일자를 알려줘,"[{'token': '100', 'start': 17, 'column_index':...","[{'token': '폐업 일자', 'start': 28, 'column_index..."
4,seouldata_healthcare_455,Whn_0002,medium,BR02,SELECT SITE_STDT FROM SEOUL_PUBLIC_HYGIENE_BIZ...,행정동명이 망우본동인 곳의 소재지 시작일은 영업장 면적이 넓은 곳부터 보여줘,"[{'token': '망우본동', 'start': 6, 'column_index':...","[{'token': '소재지 시작일', 'start': 15, 'column_ind..."


In [ ]:
import json
db_file_path = "healthcare_dataset/Training/원천데이터/seouldata_healthcare_db_annotation.json"

with open(db_file_path, "r", encoding="utf-8") as file:
    db_data = json.load(file)

db_df = pd.DataFrame(db_data['data'])
db_df.head()

,source,db_id,table_names_original,table_names,column_names_original,column_names,column_types,foreign_keys,primary_keys
0,서울특별시,seouldata_healthcare_455,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]
1,서울특별시,seouldata_healthcare_408,[L_O_C_A_L_D_A_T_A_072211],[서울시 식품제조가공업 인허가 정보],"[[-1, *], [0, OPNSFTEAMCODE], [0, MGTNO], [0, ...","[[-1, *], [0, 개방자치단체코드], [0, 관리번호], [0, 인허가일자]...","[text, number, text, time, text, number, text,...",[],[]
2,서울특별시,seouldata_healthcare_658,[L_O_C_A_L_D_A_T_A_010110],[서울시 의료유사업 인허가 정보],"[[-1, *], [0, OPNSFTEAMCODE], [0, MGTNO], [0, ...","[[-1, *], [0, 개방자치단체코드], [0, 관리번호], [0, 인허가일자]...","[text, number, text, time, time, number, text,...",[],[]
3,서울특별시,seouldata_healthcare_686,[L_O_C_A_L_D_A_T_A_020304],[서울시 동물용의약품도매상 인허가 정보],"[[-1, *], [0, OPNSFTEAMCODE], [0, MGTNO], [0, ...","[[-1, *], [0, 개방자치단체코드], [0, 관리번호], [0, 인허가일자]...","[text, number, number, number, text, time, tim...",[],[]
4,서울특별시,seouldata_healthcare_694,[F81_OPEN_TBL],[서울시 정신건강복지센터 등록회원의 지역사회 생활 유지율],"[[-1, *], [0, STAT_YEAR], [0, AREA_NAME], [0, ...","[[-1, *], [0, 연도], [0, 지역이름], [0, 주민등록인구], [0,...","[text, time, text, number, number, number, num...",[],[]


In [ ]:
merged_df = pd.merge(df, db_df, on='db_id', how='left')
merged_df.head()

,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols,source,table_names_original,table_names,column_names_original,column_names,column_types,foreign_keys,primary_keys
0,seouldata_healthcare_455,Whr_0001,easy,BR03,SELECT ADMDNG_NM FROM SEOUL_PUBLIC_HYGIENE_BIZ...,태창위생지의 행정동 이름이 뭐야,"[{'token': '태창위생지', 'start': 0, 'column_index'...","[{'token': '행정동 이름', 'start': 7, 'column_index...",서울특별시,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]
1,seouldata_healthcare_455,Wht_0001,medium,BR04,"SELECT UPSO_SITE_TELNO, TRDP_AREA FROM SEOUL_P...",2010년에 폐업한 영업장의 면적과 소재지 전화번호를 나타내줘,[],"[{'token': '영업장의 면적', 'start': 11, 'column_ind...",서울특별시,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]
2,seouldata_healthcare_455,Hch_0001,medium,BR08,"SELECT UPSO_NM, TRDP_AREA FROM SEOUL_PUBLIC_HY...",행정동명이 용신동인 업소의 이름과 영업장 면적을 영업자 시작일이 최근인 순서대로 찾아줘,"[{'token': '용신동', 'start': 6, 'column_index': ...","[{'token': '업소의 이름', 'start': 11, 'column_inde...",서울특별시,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]
3,seouldata_healthcare_455,Whn_0001,medium,BR02,SELECT DCB_YMD FROM SEOUL_PUBLIC_HYGIENE_BIZ W...,영업장 면적이 50 이하이거나 100 이상인 곳의 폐업 일자를 알려줘,"[{'token': '100', 'start': 17, 'column_index':...","[{'token': '폐업 일자', 'start': 28, 'column_index...",서울특별시,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]
4,seouldata_healthcare_455,Whn_0002,medium,BR02,SELECT SITE_STDT FROM SEOUL_PUBLIC_HYGIENE_BIZ...,행정동명이 망우본동인 곳의 소재지 시작일은 영업장 면적이 넓은 곳부터 보여줘,"[{'token': '망우본동', 'start': 6, 'column_index':...","[{'token': '소재지 시작일', 'start': 15, 'column_ind...",서울특별시,[SEOUL_PUBLIC_HYGIENE_BIZ],[서울시 기타 위생용품 제조업 현황],"[[-1, *], [0, CGG_CODE], [0, SNT_COB_CODE], [0...","[[-1, *], [0, 시군구코드], [0, 업종코드], [0, 년도], [0, ...","[text, number, number, time, number, text, tim...",[],[]


In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import GPT2LMHeadModel
import os

# Hugging Face에서 SKT KoGPT2 불러오기
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


---
Tokenizing

1. Query는 띄어쓰기 단위로 분할

2. column_names_original에서 컬럼 이름만 추출하여 추가

3. table_names_original: _ 단위로 분할하여 워드 단위로 추가

4. question은 subword 단위로 추출하여 추가

5. SQL 및 특수 토큰 추가
---





In [ ]:
make_vocab = False

if make_vocab == True:
  final_vocab = set()

  for _, row in merged_df.iterrows():
      # 1. Query는 띄어쓰기 단위로 분할하여 추가 (단, Query의 ''는 제거 후 추가)
      query_tokens = row['query'].split()
      final_vocab.update([token.replace("'", "") for token in query_tokens])
      if _ == 0:
          final_vocab.add("'")

      # 2. column_names_original에서 컬럼 이름만 추출하여 추가
      column_names = [col[1] for col in row['column_names_original'] if isinstance(col[1], str)]
      final_vocab.update(column_names)

      # 3. table_names_original: _ 단위로 분할하여 워드 단위로 추가
      for table_name in row['table_names_original']:
          final_vocab.update(table_name.split('_'))

      # 4. question은 subword 단위로 추출하여 추가
      tokens = tokenizer.tokenize(row['utterance'])
      final_vocab.update([token for token in tokens if token not in tokenizer.vocab])

  # 2. SQL 키워드 추가 (대문자 변환 후, 사전에 없는 것만 추가)
  CLAUSE_KEYWORDS = ['select', 'from', 'where', 'group', 'order', 'limit', 'intersect', 'union', 'except']
  JOIN_KEYWORDS = ['join', 'on', 'as']
  WHERE_OPS = ['not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists']
  UNIT_OPS = ['none', '-', '+', "*", '/']
  AGG_OPS = ['none', 'max', 'min', 'count', 'sum', 'avg']

  sql_keywords = [keyword.upper() for keyword in CLAUSE_KEYWORDS + JOIN_KEYWORDS + WHERE_OPS + UNIT_OPS + AGG_OPS]
  final_vocab.update([keyword for keyword in sql_keywords if keyword not in tokenizer.vocab])

  # 3. 특수 토큰 추가
  special_tokens = {
      "pad_token": "<pad>",
      "bos_token": "<sos>",
      "eos_token": "<eos>",
      "unk_token": "<unk>",
      "sep_token": "<sep>"
  }
  tokenizer.add_special_tokens(special_tokens)

  # 4. 최종 단어 목록을 토크나이저에 추가
  new_tokens = list(final_vocab - set(tokenizer.vocab))
  num_added_toks = tokenizer.add_tokens(new_tokens)
  print("추가된 새로운 단어 수:", num_added_toks)

  # 5. 모델 임베딩 레이어를 최종 단어 사전 크기에 맞춰 확장
  model.resize_token_embeddings(len(tokenizer))

  # 6. 토크나이저를 Google Drive에 저장
  drive_save_path = "/content/drive/MyDrive/Colab Notebooks/Text2SQL/text_to_sql_tokenizer"
  tokenizer.save_pretrained(drive_save_path)

  print(f"모델 임베딩 크기: {model.get_input_embeddings().weight.size()}")
  print(f"토크나이저 길이: {len(tokenizer)}")

# 추가된 새로운 단어 수: 9003
# 모델 임베딩 크기: torch.Size([60207, 768])
# 토크나이저 길이: 60207

In [ ]:
# 출력 전용 사전 생성 시, run

make_output_vocab = False

if make_output_vocab == True:
  output_vocab = set()

  # 각 행을 순회하면서 출력 전용 vocabulary에 단어 추가
  for _, row in merged_df.iterrows():

      # Input vocab과 동일하게 전처리
      # 1. Query를 띄어쓰기 단위로 분할 후 추가
      query_tokens = row['query'].split()
      output_vocab.update([token.replace("'", "") for token in query_tokens])

      # 2. 'column_names_original'에서 컬럼 이름만 추출하여 추가
      column_names = [col[1] for col in row['column_names_original'] if isinstance(col[1], str)]
      output_vocab.update(column_names)

      # 3. 테이블 이름들을 '_'로 분할한 후, 각 부분 단어 추가
      for table_name in row['table_names_original']:
          output_vocab.update(table_name.split('_'))

      # 4. 질문(utterance)을 subword 단위로 토크나이즈하고, 기존 tokenizer에 없는 토큰만 output_vocab에 추가
      tokens = tokenizer.tokenize(row['utterance'])
      output_vocab.update([token for token in tokens if token not in tokenizer.vocab])

  output_vocab_list = sorted(list(output_vocab))  # 정렬하여 순서 유지
  output_vocab_dict = {token: idx + 4 for idx, token in enumerate(output_vocab_list)}  # 특수 토큰을 위해 인덱스를 4부터 시작

  # 특수 토큰
  output_vocab_dict["<PAD>"] = 0
  output_vocab_dict["<SOS>"] = 1
  output_vocab_dict["<EOS>"] = 2
  output_vocab_dict["<UNK>"] = 3

  # output_vocab_dict을 파일로 저장
  with open('output_vocab_with_index.txt', 'w') as f:
      for token, idx in output_vocab_dict.items():
          f.write(f"{idx}\t{token}\n")

In [ ]:
# 특수 토큰 ID 확인
print("PAD Token ID:", tokenizer.pad_token_id)
print("SOS Token ID:", tokenizer.bos_token_id)
print("EOS Token ID:", tokenizer.eos_token_id)
print("UNK Token ID:", tokenizer.unk_token_id)

PAD Token ID: 3
SOS Token ID: 51201
EOS Token ID: 51202
UNK Token ID: 5


# 02. Data Loader

In [ ]:
from transformers import GPT2Tokenizer, BertTokenizer

# GPT-2 토크나이저 로드
origin_tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
maked_tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Text2SQL/text_to_sql_tokenizer")

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(maked_tokenizer)

# 추가된 새로운 단어 수: 9003
# 모델 임베딩 크기: torch.Size([60207, 768])
# 토크나이저 길이: 60207

In [ ]:
# 앞서 추가된 Vocab 내에서 output vocab 생성 코드

# 원하는 인덱스 범위 설정
start_range_1 = 0
end_range_1 = 8
start_range_2 = 51200
end_range_2 = 60206

# output_vocab 딕셔너리 생성: 0부터 새로운 인덱스를 부여
output_vocab = {}
new_index = 0

# 첫 번째 범위 (0~8)
for i in range(start_range_1, end_range_1 + 1):
    token = maked_tokenizer.convert_ids_to_tokens(i)
    output_vocab[token] = new_index
    new_index += 1

# 두 번째 범위 (51200~60206)
for i in range(start_range_2, end_range_2 + 1):
    token = maked_tokenizer.convert_ids_to_tokens(i)
    output_vocab[token] = new_index
    new_index += 1

# output_vocab 파일로 저장
import json
with open("output_vocab_reindexed.json", "w") as f:
    json.dump(output_vocab, f, ensure_ascii=False, indent=4)

print("output_vocab 크기:", len(output_vocab))

output_vocab 크기: 9016


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextToSQLDataset(Dataset):
    def __init__(self, dataframe, tokenizer, output_vocab, max_length=128):
        self.dataframe = dataframe
        self.output_vocab = output_vocab
        self.tokenizer = tokenizer
        self.max_length = max_length

        # 특수 토큰 IDs
        self.sos_token_id = tokenizer.convert_tokens_to_ids("<sos>")
        self.sep_token_id = tokenizer.convert_tokens_to_ids("<sep>")
        self.eos_token_id = tokenizer.convert_tokens_to_ids("<eos>")
        self.pad_token_id = tokenizer.convert_tokens_to_ids("<pad>")

        # Output vocab에서 특수 토큰 인덱스 (출력용)
        self.output_sos_token_id = output_vocab.get("<sos>")
        self.output_pad_token_id = output_vocab.get("<sep>")
        self.output_eos_token_id = output_vocab.get("<eos>")
        self.output_pad_token_id = output_vocab.get("<pad>")

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        # 1. Utterance 토큰화 (서브워드 단위)
        utterance_tokens = self.tokenizer.tokenize(row['utterance'])
        utterance_ids = self.tokenizer.convert_tokens_to_ids(utterance_tokens)

        # 2. Column Names 토큰화 (컬럼 이름만 추출, <sep>으로 구분)
        column_names = [col[1] for col in row['column_names_original'] if isinstance(col[1], str)]
        column_name_ids = []
        for col_name in column_names:
            col_name_tokens = self.tokenizer.tokenize(col_name)
            column_name_ids.extend(self.tokenizer.convert_tokens_to_ids(col_name_tokens) + [self.sep_token_id])

        # 3. Table Names 토큰화 ('_' 단위로 분할, <sep> 없이 연결)
        table_names = row['table_names_original']
        table_name_ids = []
        for table_name in table_names:
            table_name_tokens = []
            for part in table_name.split('_'):
                table_name_tokens.extend(self.tokenizer.tokenize(part))
            table_name_ids.extend(self.tokenizer.convert_tokens_to_ids(table_name_tokens))

        # 4. 전체 입력 ID 리스트 구성: <sos> + utterance + <sep> + column_names + <sep> + table_names + <eos>
        input_ids = [self.sos_token_id] + utterance_ids + [self.sep_token_id] + column_name_ids + table_name_ids + [self.eos_token_id]

        # padding : max_length보다 길 경우 자르고, 짧으면 <pad> 토큰으로 패딩
        input_ids = input_ids[:self.max_length] + [self.pad_token_id] * (self.max_length - len(input_ids))

        # 5. Query 토큰화
        query_tokens = row['query'].split()
        query_ids = [self.output_sos_token_id] + [self.output_vocab.get(token, self.output_vocab["<unk>"]) for token in query_tokens] + [self.output_eos_token_id]
        query_ids = query_ids[:self.max_length] + [self.output_pad_token_id] * (self.max_length - len(query_ids))

        # Attention Mask 생성
        attention_mask = [1 if token != self.pad_token_id else 0 for token in input_ids]

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(query_ids, dtype=torch.long)
        }

In [ ]:
# output_vocab 불러오기 (output_vocab을 JSON 파일로 저장했다는 가정)
with open("output_vocab_reindexed.json", "r") as f:
    output_vocab = json.load(f)

dataset = TextToSQLDataset(dataframe=merged_df, tokenizer=maked_tokenizer, output_vocab=output_vocab, max_length=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for batch in dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    print("Input IDs:", input_ids.shape)
    print(input_ids)
    print("Attention Mask:", attention_mask.shape)
    print(attention_mask)
    print("Labels:", labels.shape)
    print(labels)
    break  # 첫 배치만 출력

Input IDs: torch.Size([16, 128])
tensor([[51201,   499,     5,  ..., 56203, 51203, 59779],
        [51201,   501,     5,  ..., 58025, 51203, 59432],
        [51201,   392, 51474,  ..., 53328, 51203, 57143],
        ...,
        [51201,   501,     5,  ..., 51203, 52151, 51203],
        [51201,   501,     5,  ..., 55653, 54591, 54159],
        [51201, 53198,   501,  ...,     3,     3,     3]])
Attention Mask: torch.Size([16, 128])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
Labels: torch.Size([16, 128])
tensor([[  10,  797,    5,  ...,    3,    3,    3],
        [  10,  797, 1570,  ...,    3,    3,    3],
        [  10,  797, 7363,  ...,    3,    3,    3],
        ...,
        [  10,  797, 6255,  ...,    3,    3,    3],
        [  10,  797, 8370,  ...,    3,    3,    3],
        [  10,  797, 1505,  ...,    3,    3,  

# 03. Build Model

In [ ]:
import os

# 체크포인트를 저장할 경로 설정
checkpoint_dir = "./checkpoints_ver1"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
from transformers import AdamW
import torch
from transformers import GPT2LMHeadModel

# koGPT2 모델 로드 및 Output vocab size에 맞게 Layer 변환
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(maked_tokenizer))
model.lm_head = nn.Linear(768, len(output_vocab)).to(device)

print("Output vocab size:", len(output_vocab))
print("Model lm_head output features:", model.lm_head.out_features)

model.to(device)

Output vocab size: 9016
Model lm_head output features: 9016
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(60207, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=9016, bias=True)
)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(60207, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=9016, bias=True)
)

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = AdamW(model.parameters(), lr=0.0001)
num_epochs = 10  # 에포크 수 설정
save_interval = 100  # 체크포인트 저장 간격 (스텝 수)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # 주기적으로 체크포인트 저장
        if (step + 1) % save_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}_step_{step + 1}")
            model.save_pretrained(checkpoint_path)
            tokenizer.save_pretrained(checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")

        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

    # 에포크가 끝날 때마다 체크포인트 저장
    epoch_checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}")
    model.save_pretrained(epoch_checkpoint_path)
    tokenizer.save_pretrained(epoch_checkpoint_path)
    print(f"Checkpoint saved at {epoch_checkpoint_path}")

# 04. Predict

In [ ]:
import json

# Validation set load
val_file_path = "healthcare_dataset/Validation/라벨링데이터/TEXT_NL2SQL_label_seouldata_healthcare.json"
with open(val_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
val_df = pd.DataFrame(data['data'])

val_db_file_path = "healthcare_dataset/Validation/원천데이터/seouldata_healthcare_db_annotation.json"
with open(val_db_file_path, "r", encoding="utf-8") as file:
    db_data = json.load(file)
val_db_df = pd.DataFrame(db_data['data'])

val_merged_df = pd.merge(val_df, val_db_df, on='db_id', how='left')
val_merged_df.head()

,db_id,utterance_id,hardness,utterance_type,query,utterance,values,cols,source,table_names_original,table_names,column_names_original,column_names,column_types,foreign_keys,primary_keys
0,seouldata_healthcare_733,Whr_3005,easy,BR03,SELECT DUTYADDR FROM TB_PHARMACY_OPERATE_INFO ...,3층메디칼약국의 주소를 알려줘,"[{'token': '3층메디칼약국', 'start': 0, 'column_inde...","[{'token': '주소', 'start': 9, 'column_index': 2}]",서울특별시,[TB_PHARMACY_OPERATE_INFO],[서울시 약국 운영시간 정보],"[[-1, *], [0, HPID], [0, DUTYADDR], [0, DUTYNA...","[[-1, *], [0, 약국아이디], [0, 주소], [0, 약국명], [0, 대...","[text, text, text, text, text, number, number,...",[],[]
1,seouldata_healthcare_733,Whr_3006,medium,BR03,"SELECT DUTYNAME, DUTYADDR FROM TB_PHARMACY_OPE...",작업 일시가 2021년인 약국의 이름과 주소를 찾아줘,[],"[{'token': '약국의 이름', 'start': 14, 'column_inde...",서울특별시,[TB_PHARMACY_OPERATE_INFO],[서울시 약국 운영시간 정보],"[[-1, *], [0, HPID], [0, DUTYADDR], [0, DUTYNA...","[[-1, *], [0, 약국아이디], [0, 주소], [0, 약국명], [0, 대...","[text, text, text, text, text, number, number,...",[],[]
2,seouldata_healthcare_733,Whr_3007,medium,BR03,SELECT DUTYADDR FROM TB_PHARMACY_OPERATE_INFO ...,약국 이름이 일곱 글자인 곳의 주소가 뭐야,[],"[{'token': '주소', 'start': 17, 'column_index': 2}]",서울특별시,[TB_PHARMACY_OPERATE_INFO],[서울시 약국 운영시간 정보],"[[-1, *], [0, HPID], [0, DUTYADDR], [0, DUTYNA...","[[-1, *], [0, 약국아이디], [0, 주소], [0, 약국명], [0, 대...","[text, text, text, text, text, number, number,...",[],[]
3,seouldata_healthcare_733,Whn_2800,medium,BR02,"SELECT DUTYTIME1C, DUTYTIME4C FROM TB_PHARMACY...",이름이 가로 시작하는 약국의 월요일 진료 시간과 목요일 진료 시간을 보여줘,[],"[{'token': '월요일 진료 시간', 'start': 16, 'column_i...",서울특별시,[TB_PHARMACY_OPERATE_INFO],[서울시 약국 운영시간 정보],"[[-1, *], [0, HPID], [0, DUTYADDR], [0, DUTYNA...","[[-1, *], [0, 약국아이디], [0, 주소], [0, 약국명], [0, 대...","[text, text, text, text, text, number, number,...",[],[]
4,seouldata_healthcare_733,Whn_2801,medium,BR02,"SELECT DUTYTIME1C, DUTYTIME2C FROM TB_PHARMACY...",서울시에 있는 약국 중 가양플러스약국의 월요일과 화요일 진료 시간을 알려줘,"[{'token': '가양플러스약국', 'start': 13, 'column_ind...","[{'token': '화요일 진료 시간', 'start': 27, 'column_i...",서울특별시,[TB_PHARMACY_OPERATE_INFO],[서울시 약국 운영시간 정보],"[[-1, *], [0, HPID], [0, DUTYADDR], [0, DUTYNA...","[[-1, *], [0, 약국아이디], [0, 주소], [0, 약국명], [0, 대...","[text, text, text, text, text, number, number,...",[],[]


In [ ]:
import re
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

val_dataset = TextToSQLDataset(dataframe=val_merged_df, tokenizer=maked_tokenizer, output_vocab=output_vocab, max_length=128)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

checkpoint_path = "./checkpoints_ver1/checkpoint_epoch_5_step_300"
model = GPT2LMHeadModel.from_pretrained(checkpoint_path, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# 테스트 시에도 lm_head 레이어를 output_vocab 크기에 맞춰 조정
model.resize_token_embeddings(len(maked_tokenizer))
model.lm_head = nn.Linear(768, len(output_vocab)).to(device)
print(model)

# 모델을 평가 모드로 설정
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Exact Set Match without Value 계산을 위한 변수 초기화
correct_predictions = 0
total_predictions = 0

# output_vocab 기반 디코딩 함수 정의
def query_decode(tensor, index_to_token):
    decoded_tokens = [index_to_token.get(idx.item(), "<unk>") for idx in tensor]
    return " ".join(decoded_tokens)

# JSON 파일에서 output_vocab 불러오기 및 인덱스-토큰 매핑 생성
with open("output_vocab_reindexed.json", "r") as f:
    output_vocab = json.load(f)
index_to_token = {int(index): token for token, index in output_vocab.items()}

def remove_values_from_query(query):
    """
    주어진 SQL 쿼리에서 숫자와 문자열 값을 제거하여 쿼리 구조만 남깁니다.
    """
    # 숫자와 문자열 값을 정규식으로 제거
    query = re.sub(r"\'[^\']*\'", "''", query)  # 문자열 값 제거 (예: '망우본동' → '')
    query = re.sub(r"\b\d+\b", "0", query)       # 숫자 값 제거 (예: 50 → 0)
    return query.strip().lower()  # 대소문자 무시를 위해 소문자로 변환

# Validation 루프
for batch in tqdm(val_dataloader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)  # 정답 쿼리

    # 모델로 예측 쿼리 생성
    with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      logits = outputs.logits
      max_indices = torch.argmax(logits, dim=-1)

    # 예측 쿼리와 정답 쿼리 비교
    # 예측 쿼리 디코딩 (output_vocab 사용) 및 값 제거
    predicted_sql_query = query_decode(max_indices[0], index_to_token)
    predicted_sql_query_cleaned = remove_values_from_query(predicted_sql_query)

    # 정답 쿼리 디코딩 (output_vocab 사용) 및 값 제거
    true_sql_query = query_decode(labels[0], index_to_token)
    true_sql_query_cleaned = remove_values_from_query(true_sql_query)

    # 개별 예측과 정답 쿼리 출력
    print(f"Predicted SQL Query    : {predicted_sql_query}")
    print(f"Cleaned Predicted Query: {predicted_sql_query_cleaned}")
    print(f"True SQL Query         : {true_sql_query}")
    print(f"Cleaned True Query     : {true_sql_query_cleaned}")
    print("----")

    # Exact Set Match without Value 비교
    if predicted_sql_query_cleaned == true_sql_query_cleaned:
        correct_predictions += 1
    total_predictions += 1
    print('-'*50)

# 정확도 계산
accuracy = correct_predictions / total_predictions
print(f"Validation EM without Value Accuracy: {accuracy * 100:.2f}%")